In [1]:
# Team: Yuhung Sun, Jinghao Ye
import cv2
import numpy as np

In [2]:
# detect skin
def skin_detect(src):
    # args: src The source color image
    # returns: dst: The destination grayscale image where skin pixels are colored white and the rest are colored black
    
    # get the grayscale image
    src_gray = np.zeros(np.shape(src)[:-1], dtype=np.uint8)
    # blur the image
    dst = cv2.blur(src_gray, (3, 3))
    # set mask
    # R > 95, G > 40, B > 20
    mask = np.logical_and.reduce((src[:,:,2] > 95, src[:,:,1] > 40, src[:,:,0] > 20))
    dst[mask] = 255
    
    return dst

# find max contour
def contour_detect(img):
    # get skin image
    skin_img = skin_detect(img)
    # find contours
    contours, hierarchy = cv2.findContours(skin_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # create all black colored image as a canvass
    contour_output = cv2.cvtColor(np.zeros(np.shape(img)[:-1], dtype='uint8'), cv2.COLOR_GRAY2BGR)
    
    max_contour = None
    # find largest contour
    if len(contours) > 0:
        #print("The number of contours detected is: ", len(contours))
        max_id = max(enumerate(contours), key=lambda x : cv2.contourArea(x[1]))[0]
        max_size = cv2.contourArea(contours[max_id])
        boundrec = cv2.boundingRect(contours[max_id])
        
        max_contour = contours[max_id]
        # draw contour and inside area
        cv2.drawContours(contour_output, contours, max_id, (255, 0, 0), cv2.FILLED, 8)
        cv2.drawContours(contour_output, contours, max_id, (0, 0, 255), 1, 8)
        # draw the convexhull
        # cv2.convexHull(points[, hull[, clockwise[, returnPoints]]]) → hull
        hull = cv2.convexHull(max_contour)
        if len(hull) > 3:
            # draw the hull
            for i in range(len(hull)):
                cv2.line(contour_output, tuple(hull[i][0]), tuple(hull[(i+1) % len(hull)][0]), (120, 120, 255), 2)

            # draw rectangle for the largest contour
            cv2.rectangle(contour_output, boundrec, (0, 255, 0), 1, 8, 0)
        
    return max_contour, contour_output

# find the gesture
def gesture_detect(img, img_contour):  #->img to get size of canvas
    # create all black colored image as a canvass
    output = cv2.cvtColor(np.zeros(np.shape(img)[:-1], dtype='uint8'), cv2.COLOR_GRAY2BGR)
    # compare contours
    contour_compare = []
    for i in range(len(sample_contour)):
        contour_compare.append(cv2. matchShapes(img_contour, sample_contour[i], 1, 0.0))
    
    min_index = -1
    if min(contour_compare) < detect_thres:
        for i in range(len(sample_contour)):
            if abs(contour_compare[i] - min(contour_compare)) < 1e-5:
                #print(i + 1, " ", contour_compare)
                min_index = i
                break
        # draw the contour
        # show the number of fingers
        text = ""
        if i == 0:
            text = "one"
        elif i == 1:
            text = "two"
        elif i == 2:
            text = "three"
        elif i == 3:
            text = "four"
        elif i == 4:
            text = "five"
        elif i == 5:
            text = "thumb up"
        elif i == 6:
            text = "fist"
            
        # draw text and contours
        cv2.putText(output, text, (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.drawContours(output, [img_contour], 0, (255, 0, 0), cv2.FILLED, 8)
        cv2.drawContours(output, [img_contour], 0, (230, 255, 120), 2, 8)
        
    return output



In [3]:
# global variables
# samples' max contours
sample_contour = []
one_con,_ = contour_detect(cv2.imread('images/1.jpg'))
two_con,_ = contour_detect(cv2.imread('images/2.jpg'))
three_con,_ = contour_detect(cv2.imread('images/3.jpg'))
four_con,_ = contour_detect(cv2.imread('images/4.jpg'))
five_con,_ = contour_detect(cv2.imread('images/5.jpg'))
thumbup_con,_ = contour_detect(cv2.imread('images/thumb_up.jpg'))
fist_con,_ = contour_detect(cv2.imread('images/fist.jpg'))

sample_contour = [one_con, two_con, three_con, four_con, five_con, thumbup_con, fist_con]
# detect threshold
detect_thres = 0.2

In [4]:
# reading a stream of images from a webcamera and displaying the video
cap = cv2.VideoCapture(0)

# if not successful, exit program
if not cap.isOpened():
    print("Cannot open the video cam")
    sys.exit()

# create a window of webcamera
cv2.namedWindow("WebCamera", cv2.WINDOW_AUTOSIZE)
# create a window of interest part
cv2.namedWindow("Interest", cv2.WINDOW_AUTOSIZE)
# create a window of contour
cv2.namedWindow("Contour", cv2.WINDOW_AUTOSIZE)
# create a window of gesture
cv2.namedWindow("Gesture", cv2.WINDOW_AUTOSIZE)

# location variables
x_0 = 300
y_0 = 100
x_1 = 500
y_1 = 300
# line thick parameter
rec_line = 2


while(1):
    # read a new frame from video
    ret, frame = cap.read()
    # if not successful, break loop
    if not ret:
        print("Cannot read a frame from video stream")
        break


    # draw the part of interest with a cyan rectangle
    cv2.rectangle(frame, (x_0, y_0), (x_1, y_1), (230, 255, 120), rec_line)

    # show the interest part in "Interest" window
    # make sure the rectangle line of interest part is outside the window
    interest = frame[y_0 + (rec_line + 1) : y_1 - (rec_line + 1), x_0 + (rec_line + 1) : x_1 - (rec_line + 1)] 
    # filter the skin
    interest_dst = skin_detect(interest)
    max_contour, interest_contour = contour_detect(interest)
    gesture_contour = gesture_detect(interest, max_contour)
    #interest_defect = defect_detect(max_contour, interest)
    # show windows
    cv2.imshow("WebCamera", frame)
    cv2.imshow("Interest", interest_dst)
    cv2.imshow("Contour", interest_contour)
    cv2.imshow("Gesture", gesture_contour)
    # wait for 'esc' key press for 30ms. If 'esc' key is pressed, break loop
    if cv2.waitKey(30) == 27:
        print("esc key is pressed by user")
        break


# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

esc key is pressed by user
